# Музыка больших городов

**Задача:** Исследовать поведение слушателей в двух городах — Москве и Санкт-Петербурге.

Необходимо проверить гипотезы:

- Активность слушателей зависит от дня недели;
- Предпочтения слушателей двух городов различаются в начале недели и в конце;
- Жанровные предпочтения слушателей из двух городов различны.

**План проекта:**

1. Обзор данных
2. Предобработка данных
3. Проверка гипотез

## Обзор данных

In [1]:
# импорт библиотек
import pandas as pd

In [2]:
# откроем файл
df = pd.read_csv("yandex_music_project.csv")
df.head()

,userID,Track,artist,genre,City,time,Day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Saint-Petersburg,20:28:33,Wednesday
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Moscow,14:07:09,Friday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Saint-Petersburg,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Saint-Petersburg,08:37:09,Monday
4,E2DC1FAE,Soul People,Space Echo,dance,Moscow,08:34:34,Monday


In [3]:
# общая информация о данных
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65079 entries, 0 to 65078
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0     userID  65079 non-null  object
 1   Track     63848 non-null  object
 2   artist    57876 non-null  object
 3   genre     63881 non-null  object
 4     City    65079 non-null  object
 5   time      65079 non-null  object
 6   Day       65079 non-null  object
dtypes: object(7)
memory usage: 3.5+ MB


**Выводы:** 

В датасете 7 столбцов с типом данных object.

- userid — id пользователя;
- track — название композиции;
- artist — испольнитель;
- genre — жанр;
- city — город пользователя;
- time — время начала прослушивания;
- day — день недели.

На первый взгляд в данных есть две проблемы: названия столбцов и пропущенные значения. Также нужно будет проверить на наличие дубликатов в данных.

У нас есть информация о композиции и данные, характеризующие пользователя и его поведение. В целом этих данных достаточно для проверки гипотез.ных.

## Предобработка данных

1. Исправим стиль заголовков.
2. Обработаем пропуски.
3. Найдем дубликаты.

### Стиль заголовков

In [4]:
# перечень названий столбцов таблицы
df.columns

Index(['  userID', 'Track', 'artist', 'genre', '  City  ', 'time', 'Day'], dtype='object')

In [5]:
# переименование столбцов
df = df.rename(columns={'  userID':'user_id', 
                        'Track':'track',
                        '  City  ':'city',
                        'Day':'day'})

In [6]:
df.columns

Index(['user_id', 'track', 'artist', 'genre', 'city', 'time', 'day'], dtype='object')

### Пропуски значений

In [7]:
# подсчет пропусков
df.isna().sum()

user_id       0
track      1231
artist     7203
genre      1198
city          0
time          0
day           0
dtype: int64

Пропуски есть в названиях композиций, испольнителе и жанре. 

Для нас не так важно точное название трека и имя исполнителя, поэтому достаточно заменить пропуски в этих столбцах явным обозначением.

Две из трёх гипотез связанны с жанровыми предпочтениями пользователей, поэтому пропуски в этом столбце нельзя просто заполнить заглушкой. Необходимо установить причину пропусков и подобрать наиболее подходящий способ их заполнения. Наиболее подходящий это тот, который меньше отразиться на результатах исследования. Однако, в рамках этого проекта нет возможности проверить, почему в данных появились пропуски. Поэтому придётся заполнить пропуски также явными обозначениями и оценить, насколько это скажется на расчётах.

In [8]:
# замена пропущенных значений
columns_to_replace = ['track', 'artist', 'genre']

for column in columns_to_replace:
    df[column] = df[column].fillna('unknown')

In [9]:
df.isna().sum()

user_id    0
track      0
artist     0
genre      0
city       0
time       0
day        0
dtype: int64

### Дубликаты

Избавимся от явных дубликатов и найдем неявные. Неявные могли повиться при заполнении значения столбца жанры.

In [10]:
# подсчет явных дубликатов
df.duplicated().sum()

3826

In [11]:
# удаление явных дупликатов
# удаление старых индексов и формирование новых
df = df.drop_duplicates()

In [12]:
df.duplicated().sum()

0

In [13]:
# просмотр уникальных названий жанров
df['genre'].sort_values().unique()

array(['acid', 'acoustic', 'action', 'adult', 'africa', 'afrikaans',
       'alternative', 'alternativepunk', 'ambient', 'americana',
       'animated', 'anime', 'arabesk', 'arabic', 'arena',
       'argentinetango', 'art', 'audiobook', 'author', 'avantgarde',
       'axé', 'baile', 'balkan', 'beats', 'bigroom', 'black', 'bluegrass',
       'blues', 'bollywood', 'bossa', 'brazilian', 'breakbeat', 'breaks',
       'broadway', 'cantautori', 'cantopop', 'canzone', 'caribbean',
       'caucasian', 'celtic', 'chamber', 'chanson', 'children', 'chill',
       'chinese', 'choral', 'christian', 'christmas', 'classical',
       'classicmetal', 'club', 'colombian', 'comedy', 'conjazz',
       'contemporary', 'country', 'cuban', 'dance', 'dancehall',
       'dancepop', 'dark', 'death', 'deep', 'deutschrock', 'deutschspr',
       'dirty', 'disco', 'dnb', 'documentary', 'downbeat', 'downtempo',
       'drum', 'dub', 'dubstep', 'eastern', 'easy', 'electronic',
       'electropop', 'emo', 'entehno', '

In [14]:
# устранение неявных дубликатов
duplicates = ['hip', 'hop', 'hip-hop']
name = 'hiphop'
df['genre'] = df['genre'].replace(duplicates, name)

In [15]:
df['genre'].sort_values().unique()

array(['acid', 'acoustic', 'action', 'adult', 'africa', 'afrikaans',
       'alternative', 'alternativepunk', 'ambient', 'americana',
       'animated', 'anime', 'arabesk', 'arabic', 'arena',
       'argentinetango', 'art', 'audiobook', 'author', 'avantgarde',
       'axé', 'baile', 'balkan', 'beats', 'bigroom', 'black', 'bluegrass',
       'blues', 'bollywood', 'bossa', 'brazilian', 'breakbeat', 'breaks',
       'broadway', 'cantautori', 'cantopop', 'canzone', 'caribbean',
       'caucasian', 'celtic', 'chamber', 'chanson', 'children', 'chill',
       'chinese', 'choral', 'christian', 'christmas', 'classical',
       'classicmetal', 'club', 'colombian', 'comedy', 'conjazz',
       'contemporary', 'country', 'cuban', 'dance', 'dancehall',
       'dancepop', 'dark', 'death', 'deep', 'deutschrock', 'deutschspr',
       'dirty', 'disco', 'dnb', 'documentary', 'downbeat', 'downtempo',
       'drum', 'dub', 'dubstep', 'eastern', 'easy', 'electronic',
       'electropop', 'emo', 'entehno', '

**Выводы:** 

В данных были обнаружены три проблемы: названия столбцов, пропуски и дубликаты. 

Название столбоц исправили, от дуликатов избавились. От пропусков тоже, однако пришлось пойти на некоторое допущение: пропущенные значения в столбце с информацией о жанре закрыли значением "unknown", что может сказаться на результатах исследования.

Теперь данные готовы к проверке гипотез. гипотез.

## Проверка гипотез

1. Активность слушателей зависит от дня недели.

Сравним поведение случателей из Москвы и Санкт-Петербурга. Для этого разделим их на две группы и посчитаем количество композиций, которые они пропслушали в понедельник, среду и пятницу.

2. Предпочтения слушателей двух городов различаются в начале недели и в конце.

Слушатели из Москвы и Санкт-Петербурга слушают разную музыку утром понедельника и в пятницу. Чтобы это проверить, посчитаем топ-10 композиций в двух городах в эти дни и сравним.

3. Жанровые предпочтения слушателей двух городов различны.

В Москве больше слушают поп, а в Питере — рэп. Сгруппируем данные по жанру и посчитаем количество композиций.

### Сравнение поведения пользователей

In [16]:
# подсчет прослушиваний в каждом городе
df.groupby('city')['time'].count()

city
Moscow              42741
Saint-Petersburg    18512
Name: time, dtype: int64

In [17]:
# подсчет прослушиваний в каждый из трех дней
df.groupby('day')['time'].count()

day
Friday       21840
Monday       21354
Wednesday    18059
Name: time, dtype: int64

Получается, что в Москве больше прослушиваний, чем в Питере. Это легко объяснить тем, что в Москве просто больше самих пользователей.

Также из этих данных заметно, что меньше всего прослушиваний в середине недели. Проверим как обстоят дела в кажом городе в отдельности.

In [18]:
# функция подсчета прослушиваний для заданного дня и города
def number_tracks(day, city):
    track_list = df[(df['day'] == day)&(df['city'] == city)]
    track_list_count = track_list['user_id'].count()
    return track_list_count

In [19]:
mon_msc = number_tracks('Monday', 'Moscow')
wed_msc = number_tracks('Wednesday', 'Moscow')
fri_msc = number_tracks('Friday', 'Moscow')
mon_spb = number_tracks('Monday', 'Saint-Petersburg')
wed_spb = number_tracks('Wednesday', 'Saint-Petersburg')
fri_spb = number_tracks('Friday', 'Saint-Petersburg')

In [20]:
# таблица с результатами
data = [['Moscow', mon_msc, wed_msc, fri_msc], 
        ['Saint-Petersburg', mon_spb, wed_spb, fri_spb]]
columns = ['city','monday','wednesday','friday']
table = pd.DataFrame(data = data, columns = columns)
table

,city,monday,wednesday,friday
0,Moscow,15740,11056,15945
1,Saint-Petersburg,5614,7003,5895


**Выводы:** 

Обнаружили разницу в поведении слушателей из Москвы и Питера.

Москвичи чаще слушают музыку в понедельник и в пятницу. А вот питербуржцы больше слушают по средам.

Первая гипотеза подтверждается.

### Музыка в начале и в конце недели

In [21]:
# получение таблицы moscow_general из тех строк таблицы df, 
# для которых значение в столбце 'city' равно 'Moscow'
moscow_general = df[df['city'] == 'Moscow']

In [22]:
# получение таблицы spb_general из тех строк таблицы df,
# для которых значение в столбце 'city' равно 'Saint-Petersburg'
spb_general = df[df['city']=='Saint-Petersburg']

In [23]:
# функция, которая возвращает информацию о самых 
# популярных жанрах в указанный день в указанное время
def genre_weekday(df, day, time1, time2):
    genre_df = df[df['day'] == day]
    genre_df = genre_df[genre_df['time'] < time2]
    genre_df = genre_df[genre_df['time'] > time1]
    genre_df_grouped = genre_df.groupby('genre')['genre'].count()
    genre_df_sorted = genre_df_grouped.sort_values(ascending=False)
    return genre_df_sorted[:10] 

In [24]:
genre_weekday(moscow_general, 'Monday', '07:00', '11:00')

genre
pop            781
dance          549
electronic     480
rock           474
hiphop         286
ruspop         186
world          181
rusrap         175
alternative    164
unknown        161
Name: genre, dtype: int64

In [25]:
genre_weekday(spb_general, 'Monday', '07:00', '11:00')

genre
pop            218
dance          182
rock           162
electronic     147
hiphop          80
ruspop          64
alternative     58
rusrap          55
jazz            44
classical       40
Name: genre, dtype: int64

In [26]:
genre_weekday(moscow_general, 'Friday', '17:00', '23:00')

genre
pop            713
rock           517
dance          495
electronic     482
hiphop         273
world          208
ruspop         170
alternative    163
classical      163
rusrap         142
Name: genre, dtype: int64

In [27]:
genre_weekday(spb_general, 'Friday', '17:00', '23:00')

genre
pop            256
electronic     216
rock           216
dance          210
hiphop          97
alternative     63
jazz            61
classical       60
rusrap          59
world           54
Name: genre, dtype: int64

**Выводы:** 

В понедельниц утром:

- Слушатели из двух городов слушают похожую музыку. Есть отличие — в топ-10 Москвы попал жанр "world", а в топ-10 Питера — джаз и классика.
- Пропущенные значения повлияли на топ-10 Москвы — значение "unknown" заняло 10 место.

В пятницу вечером:

- Топ-10 по городам не сильно меняется. Поэтому считаем, что разница между Москвой и Питером не сильно выражена.

Вторая гипотеза подтвердилась частично.

### Жанровые предпочтения в Москве и Петербурге

In [28]:
# сгруппируем по жанру и посчитаем прослушивания треков
# Moscow
moscow_genres = (moscow_general
                 .groupby('genre')['genre'].count()
                 .sort_values(ascending = False))

moscow_genres.head(10)

genre
pop            5892
dance          4435
rock           3965
electronic     3786
hiphop         2096
classical      1616
world          1432
alternative    1379
ruspop         1372
rusrap         1161
Name: genre, dtype: int64

In [29]:
# сгруппируем по жанру и посчитаем прослушивания треков
# SPb
spb_genres = (spb_general
              .groupby('genre')['genre'].count()
              .sort_values(ascending=False))

spb_genres.head(10)

genre
pop            2431
dance          1932
rock           1879
electronic     1736
hiphop          960
alternative     649
classical       646
rusrap          564
ruspop          538
world           515
Name: genre, dtype: int64

**Выводы:**

Самый популярный жанр в Москве поп. А вот рэп популярен в равной степени между слушателями из Москвы и Петербурга.

Гипотеза подтвердилась частично.

## Итоги исследования

На основе предоставленных данных удалось прийти к следующим выводам:

1. Поведение слушателей сервиса различается в Москве и Сантк-Петербурге. Москвичи чаще слушают музыку в понедельник и в пятницу. А вот питербуржцы больше слушают по средам.

2. Музыкальные предпочтения не зависят от дня недели и от города. Есть некоторые различия по городам в понедельник: в Питере слушаю джаз и классику, в Москве — музыку жанра "world". Также заметно влияние пропусков в данных. Возможно результат мог быть другим.

3. В Москве действительно популярен жанр поп. В Питере вкусы слушателей несильно отличаются от Москвы. 